In [1]:
from kernel_submission_2 import *

/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train_df = load_train_data()

In [3]:
h = compress_to_household_level(train_df)
hh_level = h[['wall-quality','wall-material','building-materials','building-material-quality',
              'cielorazo','building-warning','building-score','building-quality',
              'r4h2','dependency','hogar_nin','pct_young_children','pct_children',
              'computer','television','tech',
              'mean-education-level','working_age_educ',
              'escolari','incomplete-primary','edjef',
              'residents-to-rooms',
              target_column]]
# hh_level = h

/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


KeyError: "['mean-education-level'] not in index"

In [ ]:
def get_valid_train_split(df, valid_class_size, random_state=123):
    v = get_balanced_data(df, valid_class_size, random_state)
    t = pd.concat([df.reset_index(), v.reset_index()]).drop_duplicates(keep=False).set_index(household_id)
    sample_min = target_table_breakdown(t)['total'].min()
    t = get_balanced_data(t, sample_min)
    return v, t

In [ ]:
v, t = get_valid_train_split(hh_level, 50, 4)
v[target_column].value_counts()

In [ ]:
# Get all from train_df except v data, split into binary for each class, get balanced data
t = pd.concat([hh_level.reset_index(), v.reset_index()]).drop_duplicates(keep=False).set_index(household_id)
t[target_column].value_counts()

In [ ]:
t_hh = t #compress_to_household_level(t)
v_hh = v #compress_to_household_level(v)

In [ ]:
is_1 = get_balanced_data(convert_to_binary_targets(t_hh, 1))
is_2 = get_balanced_data(convert_to_binary_targets(t_hh, 2))
is_3 = get_balanced_data(convert_to_binary_targets(t_hh, 3))
is_4 = get_balanced_data(convert_to_binary_targets(t_hh, 4))

In [ ]:
p_1, c_1 = train_clf(is_1, 25, 25)
p_2, c_2 = train_clf(is_2, 25, 50)
p_3, c_3 = train_clf(is_3, 19, 25) 
p_4, c_4 = train_clf(is_4, 12, 25)

In [ ]:
X = v_hh.drop(target_column, axis=1)
y = v_hh[target_column]

In [ ]:
pred_1 = pd.DataFrame(test_clf(p_1, c_1, X)).set_index(X.reset_index()[household_id]).rename(columns={0:'0',1:'1'})
pred_2 = pd.DataFrame(test_clf(p_2, c_2, X)).set_index(X.reset_index()[household_id]).rename(columns={0:'0',1:'2'})
pred_3 = pd.DataFrame(test_clf(p_3, c_3, X)).set_index(X.reset_index()[household_id]).rename(columns={0:'0',1:'3'})
pred_4 = pd.DataFrame(test_clf(p_4, c_4, X)).set_index(X.reset_index()[household_id]).rename(columns={0:'0',1:'4'})

In [ ]:
results = pd.concat([pred_1['1'], pred_2['2'], pred_3['3'], pred_4['4']], axis=1)
preds = results.idxmax(axis=1)
p = pd.DataFrame()
p['preds'] = preds

In [ ]:
# get all from train_df where result in preds
expected = train_df[train_df[household_id].isin(preds.index)]['Target'].astype(int)
actual = train_df[train_df[household_id].isin(preds.index)].merge(p, on=household_id, how='left')['preds'].astype(int)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(expected, actual))

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(expected, actual, labels=[1,2,3,4])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 12
plt.rcParams['patch.edgecolor'] = 'k'

In [ ]:
import itertools 
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_names=['1','2','3','4']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

- Most errors are targets 3 being predicted as 1
- Followed by targets 2 being predicted as 3
- Then 3 being predicted as 2, and 2 predicted as 1